#### IAU Project Phase 1 - Exploratory Data Analysis
#### Andrej Horváth / Adam Partl
#### Share of work : 50% / 50%

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.stats as sm_stats
import scipy.stats as stats
from sklearn import preprocessing
from sklearn.impute import KNNImputer

Load data

In [ ]:
connections_df = pd.read_csv('data/connections.csv',delimiter='\t', on_bad_lines='skip')
processes_df = pd.read_csv('data/processes.csv',delimiter='\t', on_bad_lines='skip')
profiles_df = pd.read_csv('data/profiles.csv',delimiter='\t', on_bad_lines='skip')
devices_df = pd.read_csv('data/devices.csv',delimiter='\t', on_bad_lines='skip')

1) Data description
A) Analyze data structures

In [ ]:
c1 = connections_df.columns
c2 = processes_df.columns
c3 = profiles_df.columns
c4 = devices_df.columns

print("connections :", c1)
print("processes :", c2)
print("profiles :", c3)
print("devices :", c4)

From this output we can see names of the columns in our datasets - which suggests that the important datasets will be processes and connections. Profiles and devices will be less useful, since we are trying to predict mwra, we do not care in which store the device was bought or what is the owners job, birthdate... .

In [ ]:
print("\nSummary for connections:")
print(connections_df.describe())
print("\nSummary for processes:")
print(processes_df.describe())
print("\nSummary for profiles:")
print(profiles_df.describe())
print("\nSummary for devices:")
print(devices_df.describe())

From this output we can see that a summary statistics for the datasets. 
The IMEI indicates an ID of the device.
The columns in the connections dataset seem consistent.
The columns in the processes dataset suggest higher standard deviation.

In [ ]:
print("\nSummary for profiles:")
print(profiles_df.describe(exclude=np.number))
print("\nSummary for devices:")
print(devices_df.describe(exclude=np.number))

The result of profiles and devices description does not suggest that the data could be relevant to mwra

In [ ]:
print("\nInfo for connections:")
print(connections_df.info())
print("\nInfo for processes:")
print(processes_df.info())
print("\nInfo for profiles:")
print(profiles_df.info())
print("\nInfo for devices:")
print(devices_df.info())

This output provides us with information about the datasets, like the amount of non null records or variable types.
We can see that the profiles and devices contain mostly object-like data, and connections and processes contain mostly floats.
Other than that we acn also see that the more important data to us (processes and connections) do not contain missing values.

In [ ]:
merged_df = pd.merge(connections_df, processes_df, on=['imei', 'ts'], how='inner', suffixes=('_conn', '_proc'))
len(merged_df)

While attempting to join all the datasets based on the imei (shared feature) we could not get anything useful. This is due to there being multiple records with the same imei in the data. We eventually achieved a merged dataset consisting of the records from connections and processes, even though there were still multiple records of the same imei, we could combine these on the timestamp as well. In the end we created a combined dataset that contains 15439 records as opposed to the 15081 original. This combination will be put to use in the Phase 2 of the project.

B) Attribute Analysis

In [ ]:
selected_attributes_connections = ['c.android.youtube', 'c.android.gm', 'c.android.chrome', 'c.katana', 'c.updateassist'] 
selected_attributes_processes = ['p.android.settings', 'p.android.packageinstaller', 'p.android.gm', 'p.system', 'p.android.externalstorage']

In [ ]:
def plot_and_describe(df, attributes):
        for attribute in attributes:
            sns.histplot(df[attribute], kde=True)
            plt.title(f'Distribution of {attribute}')
            plt.show()
            print(f'Descriptive statistics for {attribute}:')
            print(df[attribute].describe())

In [ ]:
plot_and_describe(connections_df, selected_attributes_connections)
plot_and_describe(processes_df, selected_attributes_processes)

We chose the selected attributes based on the correlation matrix (used later), using ones that have high correlation with MWRA.
From the resulted graphs for connections, we can see that the output is often normally distributed, which suggests a low amount of extremes and not a lot of skewness.

From the resulted graphs for processes, we can see more normal distribution, suggesting that the data is distributed around the mean. This shows a fewer extremes and consistent behavior.

C) Pair analysis

In [ ]:
connections_numeric_df = connections_df.select_dtypes(include=[np.number])
processes_numeric_df = processes_df.select_dtypes(include=[np.number])

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(connections_numeric_df.corr(), annot=True, cmap="coolwarm", fmt=".2f")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.title("Correlation Heatmap for Connections Dataset")
plt.show()

In [ ]:
columns_to_exclude = ['c.UCMobile.x86', 'c.UCMobile.intl', 'c.raider', 'c.android.vending', 'imei']
filtered_connections_df = connections_numeric_df.drop(columns=columns_to_exclude)
sns.pairplot(filtered_connections_df)
plt.show()

The results of pair analysis on the connections dataset shows that there are 6 variables that have some correlation with MWRA. We then used those to create pair plots. The pair plots show us in what way are the variables related to each other. For example, with higher usage of c.android.youtube, c.katana also rises.

In [ ]:
plt.figure(figsize=(12, 8))
sns.heatmap(processes_numeric_df.corr(), annot=True, cmap="coolwarm", fmt=".1f")
plt.xticks(rotation=90)
plt.yticks(rotation=0)
plt.title("Correlation Heatmap for Processes Dataset")
plt.show()

In [ ]:
columns_to_exclude = ['p.google', 'p.olauncher', 'p.android.gms', 'p.browser.provider', 'p.process.gapps', 'p.dogalize', 'p.android.vending', 'p.gms.persistent', 'p.android.defcontainer', 'p.simulator', 'p.notifier', 'p.inputmethod.latin', 'p.katana', 'imei']
filtered_processes_df = processes_numeric_df.drop(columns=columns_to_exclude)
sns.pairplot(filtered_processes_df)
plt.show()

The results of pair analysis on the processes dataset shows that there are 5 variables that have some correlation with MWRA. We then used those to create pair plots. The pair plots show us in what way are the variables related to each other. For example, with higher usage of p.system, p.android.gm is lower.

D) Pair analysis focused on predicted attribute

In [ ]:
def analyze_correlations(df, target_column):
    correlations = df.corr()[target_column].sort_values()
    print(f"Correlations with {target_column}:\n", correlations)
    return correlations

In [ ]:
def plot_predictor_relationships(df, target_column, predictors):
    for predictor in predictors:
        sns.scatterplot(x=df[predictor], y=df[target_column])
        plt.title(f'Relation between {predictor} and {target_column}')
        plt.show()

In [ ]:
connections_correlations = analyze_correlations(connections_numeric_df, 'mwra')
connections_potential_predictors = connections_correlations.index[-5:]
plot_predictor_relationships(connections_df, 'mwra', connections_potential_predictors)

From the results of the pair analysis focused on mwra column, we can see that the corelations go up to .57. Other than that we also created some pair plots, which show that the chance of mwra being 1 is higher with the usage of c.android.youtube.

In [ ]:
processes_correlations = analyze_correlations(processes_numeric_df, 'mwra')
processes_potential_predictors = processes_correlations.index[-5:]
plot_predictor_relationships(processes_df, 'mwra', processes_potential_predictors)

The output of this pair analysis focused on mwra in processes dataset, shows similar results as the one for connections. The results show correlation ranges up to .3. The plots are also similar, and show for example that mwra is likely related to high usage of p.system.

2) Identification of problems, integration and cleaning of data
A) Identification of problems

In [ ]:
#Check missing values
print("Rows with missing values in connections:", connections_df[connections_df.isnull().any(axis=1)].shape[0])
print("Rows with missing values in processes:", processes_df[processes_df.isnull().any(axis=1)].shape[0])
print("Rows with missing values in profiles:", profiles_df[profiles_df.isnull().any(axis=1)].shape[0])
print("Rows with missing values in devices:", devices_df[devices_df.isnull().any(axis=1)].shape[0])
print("\nMissing values in profiles:")
print(profiles_df.isnull().sum())

This output shows the sum of missing values, from which we can see that the most of them are located in profiles, and we show in which columns

In [ ]:
def convert_to_numeric(df, df_name):
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                df[col] = pd.to_numeric(df[col])
            except ValueError:
                print(f"{df_name} - Non-numeric values found in column {col}")
    return df

In [ ]:
connections_df = convert_to_numeric(connections_df, "Connections")
processes_df = convert_to_numeric(processes_df, "Processes")
devices_df = convert_to_numeric(devices_df, "Devices")
profiles_df = convert_to_numeric(profiles_df, "Profiles")

Next up we viewed the non-numeric values in each dataset, indicating that processes contain a timestamp which is likely in datetime format, and the rest of non-numeric values come from the profiles and devices datasets.

In [ ]:
def generate_boxplots(df, df_name):
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot of {col} ({df_name})')
        plt.show()

In [ ]:
generate_boxplots(connections_df, "Connections")
generate_boxplots(processes_df, "Processes")

B) Missing values

In [ ]:
#set treshold as 0.85 -> 15%
#set impute method to knn or mean
def handle_missing_values(df, threshold, impute_method):
    df = df.dropna(thresh=int(len(df) * threshold), axis=1)

    if impute_method == 'mean':
        for col in df.select_dtypes(include=[np.number]).columns:
            df[col] = df[col].fillna(df[col].mean())
    elif impute_method == 'knn':
        knn_imputer = KNNImputer(n_neighbors=5)
        df[df.select_dtypes(include=[np.number]).columns] = knn_imputer.fit_transform(df.select_dtypes(include=[np.number]))
    
    return df

In [ ]:
missing_handled_connections = handle_missing_values(connections_df, 0.85, 'mean')   #This is what handling missing values would look like if the dataset had some
print("Rows with missing values in profiles:", profiles_df[profiles_df.isnull().any(axis=1)].shape[0])
m_profiles_df = handle_missing_values(profiles_df, 0.85, '') # can not replace values since they are not numeric 
print("Rows with missing values in profiles:", m_profiles_df[m_profiles_df.isnull().any(axis=1)].shape[0])

In this output we tried to treat missing values. We used 2 approaches, to drop the columns where there is more than 15% values missing, and after that try to replace the missing values if there are still some left. We do this by either Knn or mean as imputers. Unfortunately the data that contains missing values does not contain numerical values - we can not impute those.

C) Outlier Detection

In [ ]:
#method = remove / replace 
#lower_quantile = 0.3
#upper_quartile = 0.7

def handle_outliers(df, method, lower_quantile, upper_quantile):
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    ndf = df.copy()
    for col in numeric_columns:
        if col == 'mwra' or col == 'imei':
            continue
    
        lower_bound = df[col].quantile(lower_quantile) - 1.5 * stats.iqr(df[col])
        upper_bound = df[col].quantile(upper_quantile) + 1.5 * stats.iqr(df[col])

        if method == 'remove':
            ndf = ndf[(ndf[col] >= lower_bound) & (ndf[col] <= upper_bound)]
        elif method == 'replace':
            mean_value = ndf[col].mean() 
            ndf[col] = np.where(ndf[col] < lower_bound, mean_value, 
                                np.where(ndf[col] > upper_bound, mean_value, ndf[col]))
    
    return ndf

In [ ]:
rep_connections_df = handle_outliers(connections_df, 'replace', 0.3, 0.7)
rem_connections_df = handle_outliers(connections_df, 'remove', 0.3, 0.7)

rem_processes_df = handle_outliers(processes_df, 'remove', 0.3, 0.7)

In [ ]:
numeric_columns = connections_df.select_dtypes(include=['float64', 'int64']).columns
numeric_columns = numeric_columns.drop('imei')
plt.figure(figsize=(15, 10))
rep_connections_df[numeric_columns].boxplot()
plt.xticks(rotation=90) 
plt.show()

In [ ]:
numeric_columns = processes_df.select_dtypes(include=['float64', 'int64']).columns
numeric_columns = numeric_columns.drop('imei')
plt.figure(figsize=(15, 10))
rem_processes_df[numeric_columns].boxplot()
plt.xticks(rotation=90) 
plt.show()

In this output we can see how to handle outliers. There are 2 approaches we used, replacing values - if the value was below or above the specified bound, it was set to a mean value of that column. The second approach we used was to simply remove the values that were out of bounds.
For the connections dataset, we used bounds of 0.3 and 0.7 to successfully handle most of the outliers.
For the processes dataset, we used the same bounds to handle all the outliers.
We used box plots to visualize the results

#### 3. Formulation and verification of hypotheses

##### A) Formulation

__Hypothesis 2:__ c.andoroid.youtube has on average a higher value when mwra is present

Null hypothesis H0: the average is the same for mwra = 1 and mwra = 0.

Hypothesis H1: There is a significant difference between the mean of mwra = 1 and mwra = 0.

In [ ]:
sns.histplot(connections_df['c.android.youtube'])

This data contains still contains outliers. For the rest of the solution we will use data from the previous part where outliers are handeled.

In [ ]:
sns.histplot(rep_connections_df['c.android.youtube'])

Next we will split this data into two groups based on whether mwra is 1 or 0.

In [ ]:
mwra_youtube = rep_connections_df[rep_connections_df['mwra'] == 1 ]['c.android.youtube']
nomwra_youtube = rep_connections_df[rep_connections_df['mwra'] == 0]['c.android.youtube']

In [ ]:
_ = sm.ProbPlot(mwra_youtube, fit=True).qqplot(line='45')

In [ ]:
_ = sm.ProbPlot(nomwra_youtube, fit=True).qqplot(line='45')

In [ ]:
stats.shapiro(mwra_youtube)

In [ ]:
stats.shapiro(nomwra_youtube)

In this case the $p$ value is < $0.05$ so we can assume the data does not come from a normal distribution. Since the data does not come from a normal distribution we can not use the t-test and will instead use the Mann-Whiteney U-test.

In [ ]:
stats.mannwhitneyu(mwra_youtube,nomwra_youtube)

Mann-Whiteney U-test gives us a $p$ value of 0.0 which means we will reject the null hypothesis and take hypothesis 1 as correct.

The difference in mean of value c.android.youtube is when mwra = 1 and mwra = 0 is statistically significant.

In [ ]:
sms.DescrStatsW(nomwra_youtube).tconfint_mean()

In [ ]:
sms.DescrStatsW(mwra_youtube).tconfint_mean()

In [ ]:
sns.barplot(x='mwra',y='c.android.youtube',data=rep_connections_df[(rep_connections_df.mwra==1)|(rep_connections_df.mwra==0)],capsize=0.1,err_kws={'linewidth':1})

This box plot shows that the mean of c.android.youtube is significantly higher when mwra = 1.

__Hypothesis 2:__ p.android.packageinstaller has on average a lower value when mwra is present

__Null hypothesis H0:__ The mean does not significantly change between mwra = 1 and mwra = 0.

__Hypothesis H1:__ The mean is significantly different between mwra = 1 and mwra = 0.

Firts we handle outliers in the processes_df dataframe

In [ ]:
rep_processes_df = handle_outliers(processes_df, 'replace', 0.3, 0.75)

In [ ]:
sns.histplot(rep_processes_df['p.android.packageinstaller'])

In [ ]:
_=sm.ProbPlot(rep_processes_df['p.android.packageinstaller'], fit=True).qqplot(line='45')

We separeate the data based on whether mwra is equal to 1 or 0.

In [ ]:
nomwra_package = rep_processes_df[rep_processes_df['mwra'] == 0]['p.android.packageinstaller']
mwra_package = rep_processes_df[rep_processes_df['mwra'] == 1]['p.android.packageinstaller']

In [ ]:
stats.shapiro(nomwra_package)

In [ ]:
stats.shapiro(mwra_package)

The shapiro test came out with $p$ values $< 0.05$ for both cases, meaning they are not normal distribution and we cant use student's t-test. We will continue with Mann-Whiteney U-test.

In [ ]:
stats.mannwhitneyu(mwra_package,nomwra_package)

The Mann-Whiteney U-test gives us a $p$ value of $0.0$ so we reject the __Null hypothesis H0__ and embrace __H1__. There is a significant difference between the mean of mwra = 1 and mwra = 0.

In [ ]:
sms.DescrStatsW(mwra_package).tconfint_mean()

In [ ]:
sms.DescrStatsW(nomwra_package).tconfint_mean()

In [ ]:
sns.barplot(x='mwra',y='p.android.packageinstaller',data=rep_processes_df[(rep_processes_df.mwra==1)|(rep_processes_df.mwra==0)],capsize=0.1,err_kws={'linewidth':1})

This plot shows that when mwra = 0 the mean is significantly higher compared to mwra = 1 and our hypothesis was correct.

##### B) Statistical power

First we create a function to calculate __Cohen's d__.

In [ ]:
def cohen_d(x1, x2):
    nx1 = len(x1)
    nx2 = len(x2)
    s = np.sqrt(((nx1-1) * np.std(x1, ddof=1)**2 + (nx2-1) * np.std(x2, ddof=1)**2) / (nx1 + nx2 - 2))
    return (np.abs(np.mean(x1) - np.mean(x2))) / s

We calculate cohen_d for values in $c.android.youtube$ and calculate the Statistical power.

In [ ]:
c_youtube = cohen_d(mwra_youtube,nomwra_youtube)
sm_stats.power.tt_ind_solve_power(c_youtube, len(mwra_youtube), 0.05, None, (len(nomwra_youtube)/len(mwra_youtube)))

We do the same for values in $p.android.packageinstaller$.

In [ ]:
c_package = cohen_d(mwra_package,nomwra_package)
sm_stats.power.tt_ind_solve_power(c_youtube, len(mwra_package), 0.05, None, (len(nomwra_package)/len(mwra_package)))

In both cases we get a __Statistical power__ of 1.0 meaning our test were certain to correctly reject the Null hypothesis.

#### Conclusion
The data is in a suitable format for further analysis and processing. Out of the 4 original datasets, we concluded that only connections and processes are going to be significant in further prediction of mwra. Our data had a lot of outliers present, which we treated accordingly, on the other hand we did not encounter any missing values in the connections or processes dataframes.
So in conclusion, we identified the issues in the data, found a way to treat the issues and the data is ready for further processing in phase 2.

Sources:

https://github.com/FIIT-IAU/IAU-course